In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json



In [2]:
source = requests.get('https://skimap.org/SkiAreas/index.xml').content
soup = BeautifulSoup(source,'xml')
areas = soup.find_all('skiArea')



In [3]:
def get_data(area_result):
    name = area_result.find('name').get_text()
    if area_result.find('georeferencing'):
        lat = area_result.find('georeferencing').attrs.get('lat', "CHECK")
        lng = area_result.find('georeferencing').attrs.get('lng', "CHECK")
    else:
        lat = "NoGEO"
        lng = "NoGEO"
    return name, lat, lng


for_df = []

for a in areas:
    name, lat, lng = get_data(a)
    for_df.append({
        "name": name,
        "lat": lat, 
        "lng": lng
    })
df = pd.DataFrame.from_records(for_df)
df.head()

,lat,lng,name
0,52.977947,-66.92094,Smokey Mountain Ski Club
1,53.52358,-64.020094,Northern Lights Ski Club
2,48.161777,-54.046468,White Hills
3,53.409742,-60.425457,Snow Goose Mountain (Mont Shana)
4,48.936666666667,-57.827222222222,Marble Mountain


In [4]:
df = df[['name', 'lat', 'lng']]

In [5]:
df

,name,lat,lng
0,Smokey Mountain Ski Club,52.977947,-66.92094
1,Northern Lights Ski Club,53.52358,-64.020094
2,White Hills,48.161777,-54.046468
3,Snow Goose Mountain (Mont Shana),53.409742,-60.425457
4,Marble Mountain,48.936666666667,-57.827222222222
5,Snow Valley Ski Club,53.4842024615732,-113.556683063507
6,Kinosoo Ridge,54.4400994648045,-110.019149780273
7,Marmot Basin,52.798588378277,-118.10078236972
8,Nakiska Ski Resort,50.947773770583,-115.17152563367
9,Sunshine Village,51.08611777439,-115.77390455793


In [37]:
df = df[df.lat != 'NoGEO']
df = df[df.lat != 'CHECK']

In [38]:
df = df[df.lng != 'NoGEO']
df = df[df.lng != 'CHECK']

In [39]:
len(df)

3720

In [40]:
df['lat'] = df['lat'].astype(float)
df['lng'] = df['lng'].astype(float)

In [42]:
def df_to_geojson(df, properties, lat='lat', lon='lng'):
    
    
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson
    
    
    

In [46]:
useful_columns = ['name']
geojson_dict = df_to_geojson(df, properties= useful_columns)
geojson_str = json.dumps(geojson_dict, indent=2)

In [47]:
output_filename = 'dataset.js'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(geojson_str))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson_dict['features'])))

3720 geotagged features saved to file


In [12]:
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

from sqlalchemy import inspect
import sqlite3 
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date

Base = declarative_base()

In [15]:
class ski_resorts(Base):
    __tablename__ = "ski_resorts"
    name = Column(String, primary_key=True)
    lat = Column(Float)
    lng = Column(Float)
    

C:\Users\hanna\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.ski_resorts, and will be replaced in the string-lookup table.
  item.__name__


In [16]:
engine = create_engine("sqlite:///ski_resorts.sqlite")
Base.metadata.create_all(engine)
Base = automap_base()
Base.prepare(engine, reflect=True)
inspector = inspect(engine)


In [17]:
ski_resorts = Base.classes.ski_resorts
session = Session(engine)

In [18]:
Base.classes.items()

[('ski_resorts', sqlalchemy.ext.automap.ski_resorts)]

In [19]:
Base.classes.keys()

['ski_resorts']